# Unit Simulation Experimental Notebook

In [1]:
from IPython.display import HTML
import numpy as np
import pandas as pd
from matplotlib import animation
import matplotlib.pyplot as plt
%matplotlib inline
# our simulator
import battlesim as bsm

### Trial 1. 50 Clone Troopers vs. 80 B1 battledroids

Using Random AI selection for targets.

In [10]:
trial1 = bsm.Battle("datasets/starwars-clonewars.csv")
# define 100 battledroids, 50 clones
trial1.create_army([("B1 battledroid", 80), ("Clone Trooper", 20), ("Clone Trooper", 20)])
# battle droids near (0, 0), clones near (10, 10)
trial1.apply_position([
    dict(name="normal", scale=3.),
    dict(name="gaussian", loc=10.),
    dict(name="gaussian", x_loc=7., y_loc=5.),
])
#init, rolling AIs now default to 'nearest'
trial1.set_ai_decision("hit_and_run")

# run a battle
F = trial1.simulate(max_step=200)

In [11]:
trial1.composition_

,unit,allegiance,n,position,init_ai,rolling_ai,decision_ai
0,b1 battledroid,CIS,80,normal,nearest,nearest,hit_and_run
1,clone trooper,Republic,20,gaussian,nearest,nearest,hit_and_run
2,clone trooper,Republic,20,gaussian,nearest,nearest,hit_and_run


In [12]:
HTML(trial1.sim_jupyter().to_jshtml())

In [5]:
trial1.sim_export("simulations/main.gif",writer="imagemagick")

## Testing Performance Profiling

In [6]:
%timeit trial1.simulate()

70.7 ms ± 3.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
